In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer # Import TF IDF 
from typing import List 
import re

In [3]:
df = pd.read_csv('mail_data.csv')
print(df.head())

  Category                                            Message
0      ham  Go until jurong point, crazy.. Available only ...
1      ham                      Ok lar... Joking wif u oni...
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...
3      ham  U dun say so early hor... U c already then say...
4      ham  Nah I don't think he goes to usf, he lives aro...


In [4]:
df['Category'] = df['Category'].map({'ham': 0, 'spam': 1})

In [5]:
df.head()

,Category,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [11]:
print(df['Message'][101])

Okay name ur price as long as its legal! Wen can I pick them up? Y u ave x ams xx


Implement `TF IDF`
- Using tokenization function to split text into individual word
    - sdfsefef

In [20]:
def compute_tfidf(documents):
    # Tokenize document
    tokenized_docs = [doc.lower().split() for doc in documents]

    # Build vocabulary
    vocab = sorted(set(word for doc in tokenized_docs for word in doc))

    # Compute term frequency
    def conpute_tf(doc_tokens):
        tf = {}
        total_terms = len(doc_tokens)
        for word in vocab:
            tf[word] = doc_token.count(word) / total_terms

        return tf

    tf_list = [compute_tf(doc) for doc in tokenized_docs]

    # Compute document frequency 
    def compute_df(docs):
        df = {}
        for word in vocab:
            df[word] = sum(1 for doc in docs if word in doc)
        return df

    df = compute_df(tokenized_docs)

    # Compute inverse document frequency (IDF)
    def compute_idf(df, N):
        idf = {}
        for word, doc_count in df.items():
            idf[word] = math.log(N / (1 + doc_count))
        return idf

    idf = compute_idf(df, len(documents))

    # Calculating TF-IDF
    def compute_tfidf(tf, idf):
        tfidf = {}
        for word in vocab:
            tdidf[word] = tf[word] * idf[word]
        return tfidf

    tfidf_list = [compute_tfidf(tf,idf) for tf in tf_list]

    result = []
    for i, tfidf in enumrate(tfidf_list):
        result.append({word: round(score, 4) for word, score in tfidf.items() if score > 0})

    return result